In [1]:
import ollama
import numpy as np
import jpt
from typing import List, Union
from jpt import load_jeopardy_dataset

In [2]:
jeopardy_data = load_jeopardy_dataset()

In [3]:
%%timeit -n 3 -r 3

e = ollama.embeddings(
    model='nomic-embed-text',
    prompt='Llamas are members of the camelid family',
)
v = e['embedding']
np.array(v).shape

29.1 ms ± 12.6 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [4]:
@jpt.retry_decorator
def nomic_embed(chunk):
    response = ollama.embeddings(
        model='nomic-embed-text',
        prompt=chunk
    )
    embedding_vector = np.array(response['embedding'], dtype='float32')
    return embedding_vector


In [5]:
def nomic_embeddings(chunks: List[str]) -> np.ndarray:
    n = len(chunks)
    m = 768
    database = np.zeros(shape=(n, m), dtype='float32')
    
    for index, chunk in enumerate(chunks):
        database[index, :] = nomic_embed(chunk)

    return database

In [6]:
movie_titles = [
    "The Shawshank Redemption",
    "The Godfather",
    "The Dark Knight",
    "Pulp Fiction",
    "Schindler's List",
    "The Lord of the Rings: The Return of the King",
    "Inception",
    "Fight Club",
    "Forrest Gump",
    "The Matrix"
]

movie_database = nomic_embeddings(movie_titles)
print(movie_database.shape)

(10, 768)


In [8]:
jeopardy_chunk_template = '''JEOPARDY QUESTION:
category: {category!r}
clue: {question}
correct response: {answer!r}'''

jeopardy_chunks = [ jeopardy_chunk_template.format(**q) for q in  jeopardy_data ]

In [10]:
def compute_embeddings():
    global jeopardy_nomic_embeddings
    jeopardy_nomic_embeddings = nomic_embeddings(jeopardy_chunks)

In [11]:
%%timeit -n 1 -r 1
compute_embeddings()

54min 40s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
jeopardy_nomic_embeddings.shape

(216930, 768)

In [13]:
from util import total_size
total_size(jeopardy_nomic_embeddings)

666408960